In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer #TfidfVectorizer: Metinleri sayısal vektörlere (embedding) dönüştürmek için kullanılır. Her kelimenin önemini ölçer.
from sklearn.metrics.pairwise import cosine_similarity #İki vektör arasındaki benzerliği (cosine similarity) hesaplar.
import numpy as np
import pickle

1. Adım: Hikayeleri ve Soruyu Tanımla

In [2]:
hikayeler = [
    "Ormanda kaybolan bir grup maceracı vardı.",
    "Küçük bir köyde herkes birbirine yardım ederdi.",
    "Bir gün deniz kenarında ilginç bir taş bulundu.",
    "Şehirde büyük bir festival düzenlendi.",
    "Yaşlı adam her sabah parkta yürüyüş yapardı.",
    "Çocuklar okul bahçesinde oyun oynuyordu.",
    "Bir köpek sahibini bulmak için uzun bir yolculuğa çıktı.",
    "Kütüphanede eski bir kitap keşfedildi.",
    "Dağ köyünde bir gece fırtına çıktı.",
    "Genç kadın yeni bir işe başladı."
]

In [3]:
sorular = [
    "Ormanda ne oldu?",
    "Köyde kimler yaşıyor?",
    "Deniz kenarında ne bulundu?",
    "Festival nerede düzenlendi?",
    "Parkta kim yürüyüş yapıyor?",
    "Okul bahçesinde kimler var?",
    "Köpek ne yaptı?",
    "Kütüphanede ne bulundu?",
    "Dağ köyünde ne oldu?",
    "Kim yeni bir işe başladı?"
]


2. Adım: Fonksiyon: En uygun hikaye ve benzerlik skorunu bul

In [ ]:
def embedding_ve_benzerlik_hesapla(hikayeler, soru):
    tum_metinler = hikayeler + [soru] # Tüm metinleri (hikayeler + soru) birleştir
    vectorizer = TfidfVectorizer() 
    
    tfidf_matrisi = vectorizer.fit_transform(tum_metinler) #fit_transform: tüm_metinler listesindeki her metni sayısal vektöre çevirir. 
    # Sonuç: (11, n) boyutunda bir matris (11 metin, n kelime).
    
    # Hikaye ve soru embeddinglerini ayır
    hikaye_embeddingleri = tfidf_matrisi[:-1] #hikaye_embeddingleri: Son satır hariç tüm satırlar (ilk 10 satır) hikaye embeddingleri olur.
    soru_embedding = tfidf_matrisi[-1] # soru_embedding: Son satır ([-1]) soru embeddingidir.
    
    # Cosine Similarity Hesaplama: Her hikaye ile sorunun embedding'i arasındaki benzerliği bulalım.
    benzerlikler = cosine_similarity(soru_embedding, hikaye_embeddingleri).flatten()
    # flatten(): Sonucu tek boyutlu diziye çevirir (10 elemanlı bir liste).

    # En yüksek benzerliğe sahip hikayeyi bulalım.
    en_yuksek_indeks = np.argmax(benzerlikler) #np.argmax: En yüksek benzerlik değerinin indeksini bulur.
    en_yuksek_benzerlik = benzerlikler[en_yuksek_indeks]
    en_uygun_hikaye = hikayeler[en_yuksek_indeks]
    return en_uygun_hikaye, en_yuksek_benzerlik

3. Tüm sorular için otomatik test ve sonuçları toplama

In [5]:
sonuclar = []
for soru in sorular:
    en_uygun_hikaye, benzerlik = embedding_ve_benzerlik_hesapla(hikayeler, soru)
    sonuclar.append({
        "soru": soru,
        "en_uygun_hikaye": en_uygun_hikaye,
        "benzerlik": benzerlik
    })

 4. Sonuçları pickle ile dosyaya kaydet


In [6]:
# Sonuçları kaydet
with open("sonuclar.pickle", "wb") as f:
    pickle.dump(sonuclar, f)
print("Tüm sorular işlendi ve sonuçlar 'sonuclar.pickle' dosyasına kaydedildi.")

Tüm sorular işlendi ve sonuçlar 'sonuclar.pickle' dosyasına kaydedildi.


5. Sonuçları tekrar yükleyip ekrana yazdırma

In [8]:
with open("sonuclar.pickle", "rb") as f:
    yuklenen_sonuclar = pickle.load(f)
    
for sonuc in yuklenen_sonuclar:
    print(f"Soru: {sonuc['soru']}")
    print(f"En uygun hikaye: {sonuc['en_uygun_hikaye']}")
    print(f"Benzerlik skoru: {sonuc['benzerlik']:.3f}")
    print("-" * 40)

Soru: Ormanda ne oldu?
En uygun hikaye: Ormanda kaybolan bir grup maceracı vardı.
Benzerlik skoru: 0.199
----------------------------------------
Soru: Köyde kimler yaşıyor?
En uygun hikaye: Küçük bir köyde herkes birbirine yardım ederdi.
Benzerlik skoru: 0.181
----------------------------------------
Soru: Deniz kenarında ne bulundu?
En uygun hikaye: Bir gün deniz kenarında ilginç bir taş bulundu.
Benzerlik skoru: 0.499
----------------------------------------
Soru: Festival nerede düzenlendi?
En uygun hikaye: Şehirde büyük bir festival düzenlendi.
Benzerlik skoru: 0.486
----------------------------------------
Soru: Parkta kim yürüyüş yapıyor?
En uygun hikaye: Yaşlı adam her sabah parkta yürüyüş yapardı.
Benzerlik skoru: 0.309
----------------------------------------
Soru: Okul bahçesinde kimler var?
En uygun hikaye: Çocuklar okul bahçesinde oyun oynuyordu.
Benzerlik skoru: 0.372
----------------------------------------
Soru: Köpek ne yaptı?
En uygun hikaye: Bir köpek sahibini bulmak